# Import packages


In [1]:
import numpy as np
import pylab as pl
from numpy import fft
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta
import math


# Load data

In [2]:
def load_data(stock_name, date_predict_start, data_range, slide_range, n_slide):
    train_data = {}
    test_data = {}
    date_predict_start = datetime.datetime.strptime(
        date_predict_start, '%Y-%m-%d')
    date_data_start_list = []
    date_predict_start_list = []
    date_predict_end_list = []
    for i in range(n_slide*2):
        date_data_start = date_predict_start - \
            relativedelta(days=+data_range)
        date_predict_end = date_predict_start + \
            relativedelta(days=+data_range)
        date_data_start_list.append(date_data_start)
        date_predict_start_list.append(date_predict_start)
        date_predict_end_list.append(date_predict_end)
        date_data_start = date_data_start + \
            relativedelta(days=+slide_range)
        date_predict_start = date_predict_start + \
            relativedelta(days=+slide_range)

    train_data_all = yf.Ticker(stock_name).history(
        start=date_data_start_list[0], end=date_predict_start_list[-1])
    test_data_all = yf.Ticker(stock_name).history(
        start=date_predict_start_list[0], end=date_predict_end_list[-1])
    test_data_all['count'] = range(len(test_data_all))
    test_data_start_list = []
    for i in range(n_slide):
        train_data['data_' + str(i)] = train_data_all.iloc[i *
                                                           slide_range:i*slide_range+data_range]
        train_data['data_' + str(i)] = train_data['data_' +
                                                  str(i)].reset_index(drop=True)
        test_data['data_' + str(i)] = test_data_all.iloc[i *
                                                         slide_range:i*slide_range+data_range]
        test_data_start_list.append(test_data['data_' + str(i)].index[0])
        test_data['data_' + str(i)] = test_data['data_' +
                                                str(i)].reset_index(drop=True)
    return train_data, test_data, test_data_all, test_data_start_list


# Data preprocessing

In [3]:
def find_data_pv_CL_function(data, pv_range):
    pd.options.mode.chained_assignment = None
    for i in data:
        pv = data[i]['Close']
        data[i]['peaks'] = pd.Series(dtype='float64')
        data[i]['valleys'] = pd.Series(dtype='float64')
        peaks = data[i]['peaks']
        valleys = data[i]['valleys']
        for idx in range(0, len(pv)):
            if idx < pv_range:
                if pv[idx] == pv.iloc[0:pv_range*2+1].max():
                    peaks.iloc[idx] = pv[idx]
                if pv[idx] == pv.iloc[0:pv_range*2+1].min():
                    valleys.iloc[idx] = pv[idx]
            if pv[idx] == pv.iloc[idx-pv_range:idx+pv_range].max():
                peaks.iloc[idx] = pv[idx]
            if pv[idx] == pv.iloc[idx-pv_range:idx+pv_range].min():
                valleys.iloc[idx] = pv[idx]
        data[i]['peaks'] = peaks
        data[i]['valleys'] = valleys
    

In [4]:
def find_data_pv_HL_function(data, pv_range):
    pd.options.mode.chained_assignment = None
    for i in data:
        p = data[i]['High']
        v = data[i]['Low']
        data[i]['peaks'] = pd.Series(dtype='float64')
        data[i]['valleys'] = pd.Series(dtype='float64')
        peaks = data[i]['peaks']
        valleys = data[i]['valleys']
        for idx in range(0, len(p)):
            if idx < pv_range:
                if p[idx] == p.iloc[0:pv_range*2+1].max():
                    peaks.iloc[idx] = p[idx]
                if v[idx] == v.iloc[0:pv_range*2+1].min():
                    valleys.iloc[idx] = v[idx]
            if p[idx] == p.iloc[idx-pv_range:idx+pv_range].max():
                peaks.iloc[idx] = p[idx]
            if v[idx] == v.iloc[idx-pv_range:idx+pv_range].min():
                valleys.iloc[idx] = v[idx]
        data[i]['peaks'] = peaks
        data[i]['valleys'] = valleys

In [5]:
def preprocessing(train_data, test_data, pv_range, pv_method):
    if pv_method == 'CL':
        find_data_pv_CL_function(train_data, pv_range)
        find_data_pv_CL_function(test_data, pv_range)
    elif pv_method == 'HL':
        find_data_pv_HL_function(train_data, pv_range)
        find_data_pv_HL_function(test_data, pv_range)


# Built Model

Get signal

In [6]:
def data_to_harmonics_function(data_stock):
    harmonics = {}
    for i in data_stock:
        harmonics[i] = {}
        # get data_stock's infomation
        data = data_stock[i]['Close']
        array_data = np.array(data)
        n_data = array_data.size
        time_data = np.arange(0, n_data)

        # detrend data
        # find linear trend in data
        Polynomial = np.polyfit(time_data, array_data, 1)
        data_notrend = array_data - Polynomial[0] * time_data    # detrended x

        # fft process
        data_freqdom = fft.fft(data_notrend, n=n_data)
        frequence = fft.fftfreq(n=n_data, d=1)
        f_positive = frequence[np.where(frequence > 0)]
        data_freqdom_positive = data_freqdom[np.where(frequence > 0)]

        # sort indexes
        indexes = list(range(f_positive.size))      # frequencies
        # sort method 1
        # indexes.sort(key = lambda i: np.absolute(frequence[i]))     # sort indexes by frequency, lower -> higher
        # sort method 2 :
        # sort indexes by amplitudes, lower -> higher
        indexes.sort(key=lambda i: np.absolute(data_freqdom[i]))
        indexes.reverse()       # sort indexes by amplitudes, higher -> lower

        # get data_all_time'size
        time_transfer = np.arange(0, 2*array_data.size)

        # mix harmonics
        for j in indexes:
            ampli = np.absolute(
                data_freqdom_positive[j]) / n_data     # amplitude
            phase = np.angle(data_freqdom_positive[j])      # phase
            harmonics[i][j] = ampli * \
                np.cos(2 * np.pi * f_positive[j] * time_transfer + phase)
    return harmonics


In [7]:
def mix_harmonics_function(harmonics, n_harm_lower_limit, n_harm_upper_limit):
    processed_signal = {}
    for i in harmonics:
        processed_signal[i] = {}
        for n_harm in range(n_harm_lower_limit, n_harm_upper_limit+1):
            mixed_harmonic = np.zeros(len(harmonics[i][0]))
            for j in range(n_harm):
                mixed_harmonic += harmonics[i][j]
            processed_signal[i][n_harm] = pd.DataFrame(
                {'Close': mixed_harmonic})
    return processed_signal


Signal processing

In [8]:
def find_signal_pv_function(signal, pv_range):
    pd.options.mode.chained_assignment = None
    for i in signal:
        for j in signal[i]:
            pv = signal[i][j]['Close']
            signal[i][j]['peaks'] = pd.Series(dtype='float64')
            signal[i][j]['valleys'] = pd.Series(dtype='float64')
            peaks = signal[i][j]['peaks']
            valleys = signal[i][j]['valleys']
            for idx in range(0, len(pv)):
                if idx < pv_range:
                    if pv[idx] == pv.iloc[0:pv_range*2+1].max():
                        peaks.iloc[idx] = pv[idx]
                    if pv[idx] == pv.iloc[0:pv_range*2+1].min():
                        valleys.iloc[idx] = pv[idx]
                if pv[idx] == pv.iloc[idx-pv_range:idx+pv_range].max():
                    peaks.iloc[idx] = pv[idx]
                if pv[idx] == pv.iloc[idx-pv_range:idx+pv_range].min():
                    valleys.iloc[idx] = pv[idx]
            signal[i][j]['peaks'] = peaks
            signal[i][j]['valleys'] = valleys


In [9]:
def find_pv_lead_function(data, processed_signal):
    for d in data:
        for p in processed_signal[d]:
            processed_signal[d][p]['pv'] = pd.Series(dtype='str')
            processing_signal = processed_signal[d][p].loc[list(data[d].index)]
            p_data = pd.DataFrame(
                {'peaks': data[d]['peaks'], 'count': range(len(data[d]))})
            p_data = p_data.drop(p_data[p_data['peaks'].isna()].index)
            p_data_count = list(p_data['count'])
            p_signal = pd.DataFrame(
                {'peaks': processing_signal['peaks'], 'count': range(len(processing_signal))})
            p_signal = p_signal.drop(p_signal[p_signal['peaks'].isna()].index)
            p_signal_list = list(p_signal['count'])
            p_lead = []
            for i in range(0, len(p_signal_list)):
                temp = []
                temp_abs = []
                temp_2 = []
                for j in range(0, len(p_data_count)):
                    temp.append((p_data_count[j] - p_signal_list[i]))
                    temp_abs.append(abs(p_data_count[j] - p_signal_list[i]))
                for k in range(0, len(temp_abs)):
                    if temp_abs[k] == min(temp_abs):
                        temp_2 = temp[k]
                p_lead.append(temp_2)
            p_signal['lead'] = p_lead

            v_data = pd.DataFrame(
                {'valleys': data[d]['valleys'], 'count': range(len(data[d]))})
            v_data = v_data.drop(v_data[v_data['valleys'].isna()].index)
            v_data_count = list(v_data['count'])
            v_signal = pd.DataFrame(
                {'valleys': processing_signal['valleys'], 'count': range(len(processing_signal))})
            v_signal = v_signal.drop(
                v_signal[v_signal['valleys'].isna()].index)
            v_signal_list = list(v_signal['count'])
            v_lead = []
            for i in range(0, len(v_signal_list)):
                temp = []
                temp_abs = []
                temp_2 = []
                for j in range(0, len(v_data_count)):
                    temp.append((v_data_count[j] - v_signal_list[i]))
                    temp_abs.append(abs(v_data_count[j] - v_signal_list[i]))
                for k in range(0, len(temp_abs)):
                    if temp_abs[k] == min(temp_abs):
                        temp_2 = temp[k]
                v_lead.append(temp_2)
            v_signal['lead'] = v_lead

            processed_signal[d][p]['lead'] = pd.Series(dtype='float64')
            processed_signal[d][p]['lead'].loc[p_signal['lead'].index] = p_signal['lead']
            processed_signal[d][p]['pv'].loc[p_signal['lead'].index] = 'peak'
            processed_signal[d][p]['lead'].loc[v_signal['lead'].index] = v_signal['lead']
            processed_signal[d][p]['pv'].loc[v_signal['lead'].index] = 'valley'


In [10]:
def build_model(train_data, n_harm_lower_limit, n_harm_upper_limit, pv_range):
    harmonics = data_to_harmonics_function(train_data)
    processed_signal = mix_harmonics_function(
        harmonics, n_harm_lower_limit, n_harm_upper_limit)
    find_signal_pv_function(processed_signal, pv_range)
    find_pv_lead_function(train_data, processed_signal)
    return harmonics, processed_signal


# Select model

In [11]:
def get_fit_error_function(processed_signal, fit_method):
    errors = {}
    error = []
    for i in processed_signal:
        errors[i] = {}
        for j in processed_signal[i]:
            signal_dropna = processed_signal[i][j].drop(
                processed_signal[i][j][processed_signal[i][j]['lead'].isna()].index)
            if fit_method == 'mean':
                error = signal_dropna['lead'].mean()
            elif fit_method == 'abs':
                error = abs(signal_dropna['lead']).mean()
            elif fit_method == 'rmse':
                mse = np.square(np.subtract(np.zeros_like(signal_dropna['lead']),signal_dropna['lead'])).mean() 
                rmse = math.sqrt(mse)
                error = rmse
            errors[i][j] = error
    return errors


In [12]:
def get_best_fit_harm_function(processed_signal, errors):
    best_error = {}
    best_fit_harm = {}
    for i in processed_signal:
        best_error[i] = pd.Series(errors[i]).abs().min()
        best_fit_harm[i] = pd.Series(errors[i]).abs().idxmin()
    return best_fit_harm, best_error


In [13]:
def get_first_lead_function(processed_signal, best_fit_harm):
    first_date = {}
    lead = {}
    pv = {}
    for i in processed_signal:
        harm = best_fit_harm[i]
        temp = processed_signal[i][harm].loc[list(
            processed_signal[i][harm]['lead'].dropna().index)[0]]
        first_date[i] = list(processed_signal[i][harm]
                             ['lead'].dropna().index)[0]
        lead[i] = temp['lead']
        pv[i] = temp['pv']
    return first_date, lead, pv


In [14]:
def select_model(processed_signal, fit_method):
    errors = get_fit_error_function(processed_signal, fit_method)
    best_fit_harm, best_error = get_best_fit_harm_function(
        processed_signal, errors)
    first_date, lead, pv = get_first_lead_function(
        processed_signal, best_fit_harm)
    return errors, best_fit_harm, best_error, first_date, lead, pv


# Evaluate Model

In [15]:
def built_result_table_function(processed_signal, test_data_start_list, lead, pv, best_error, best_fit_harm):
    result_table = pd.DataFrame(columns=[
        's_date', 't_date', 'lead', 'ans_date', 'pv', 'error', 'best_fit'])
    for i in processed_signal:
        result_table.loc[i, 'error'] = round(best_error[i], 2)
        result_table.loc[i, 'best_fit'] = best_fit_harm[i]
        result_table.loc[i, 'lead'] = lead[i]
        result_table.loc[i, 'pv'] = pv[i]
    result_table['s_date'] = test_data_start_list
    return result_table


In [16]:
def result_table_process_function(result_table, test_data_all, first_date):
    for i in result_table.index:
        t_date = test_data_all.loc[test_data_all['count'] ==
                                   test_data_all['count'].loc[result_table.loc[i, 's_date']] +
                                   first_date[i]].index[0]
        t_date = datetime.datetime.strftime(t_date, '%Y-%m-%d')
        result_table.loc[i, 't_date'] = t_date
        ans = test_data_all.loc[test_data_all['count'] == test_data_all['count']
                                .loc[result_table.loc[i, 't_date']] + result_table.loc[i, 'lead']].index[0]
        ans = datetime.datetime.strftime(ans, '%Y-%m-%d')
        result_table.loc[i, 'ans_date'] = ans


In [17]:
def compute_final_error_function(result_table):
    final_error = round(
        sum([abs(ele) for ele in result_table['lead']]) / len(result_table['lead']), 2)
    return final_error


In [18]:
def evaluate_model(processed_signal, test_data_start_list, test_data_all, best_fit_harm, best_error, first_date, lead, pv):
    result_table = built_result_table_function(
        processed_signal, test_data_start_list, lead, pv, best_error, best_fit_harm)
    result_table_process_function(result_table, test_data_all, first_date)
    final_error = compute_final_error_function(result_table)
    return result_table, final_error


# Main function

In [19]:
def main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
        n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method):

    # 1. Load data
    train_data, test_data, test_data_all, test_data_start_list = load_data(
        stock_name, date_predict_start, data_range, slide_range, n_slide)
    # 2. Preprocessing
    preprocessing(train_data, test_data, pv_range, pv_method)
    
    # 3. Build model
    harmonics, model = build_model(
        train_data, n_harm_lower_limit, n_harm_upper_limit, pv_range)
    # 4. Select model
    errors, best_fit_harm, best_error, first_date, lead, pv = select_model(
        model, fit_method)

    # 5. Evaluate model
    result_table, final_error = evaluate_model(
        model, test_data_start_list, test_data_all, best_fit_harm, best_error, first_date, lead, pv)
    print('final_error = ', final_error)
    print(result_table)
    # 6. Predict

    return harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error


In [20]:
stock_name = "^GSPC"
date_predict_start = '2019-01-01'
data_range = 20
slide_range = 5
n_slide = 10
pv_range = 2
n_harm_lower_limit = 5
n_harm_upper_limit = 5
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  0.9
           s_date      t_date lead    ans_date      pv error best_fit
data_0 2018-12-31  2019-01-03 -1.0  2019-01-02    peak  0.89        5
data_1 2019-01-08  2019-01-08  1.0  2019-01-09    peak  3.42        5
data_2 2019-01-15  2019-01-17  0.0  2019-01-17    peak  2.24        5
data_3 2019-01-23  2019-01-23  0.0  2019-01-23  valley  2.65        5
data_4 2019-01-30  2019-01-31  5.0  2019-02-07    peak  2.27        5
data_5 2019-02-06  2019-02-07  0.0  2019-02-07    peak   1.5        5
data_6 2019-02-13  2019-02-15 -1.0  2019-02-14  valley  1.54        5
data_7 2019-02-21  2019-02-25  0.0  2019-02-25    peak  0.53        5
data_8 2019-02-28  2019-02-28  0.0  2019-02-28  valley  0.65        5
data_9 2019-03-07  2019-03-11  1.0  2019-03-12  valley  0.82        5
